In [1836]:
from sklearn import linear_model as lm
from sklearn.model_selection import train_test_split
import sklearn.preprocessing as prep
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tabulate import tabulate

In [1837]:
df_prostate = pd.read_csv('prostate.data',
                 delimiter='\t',
                 index_col = 0)
df_prostate = df_prostate.iloc[:,:-1]
prostate_data = df_prostate.to_numpy().astype('float')
df_prostate.head()


,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45,lpsa
1,-0.579818,2.769459,50,-1.386294,0,-1.386294,6,0,-0.430783
2,-0.994252,3.319626,58,-1.386294,0,-1.386294,6,0,-0.162519
3,-0.510826,2.691243,74,-1.386294,0,-1.386294,7,20,-0.162519
4,-1.203973,3.282789,58,-1.386294,0,-1.386294,6,0,-0.162519
5,0.751416,3.432373,62,-1.386294,0,-1.386294,6,0,0.371564


In [1838]:
def train_val_test_split(x, y, test_percent, val_percent, random_state = None):
    rel_val_percent = np.round((val_percent)/(1-test_percent), 2)

    x_inter, x_test, y_inter, y_test = train_test_split(x, y, test_size = test_percent, random_state = random_state)
    x_train, x_val, y_train, y_val = train_test_split(x_inter, y_inter, test_size = rel_val_percent, random_state = random_state)
    
    return x_train, x_val, x_test, y_train, y_val, y_test

In [1839]:
split_data = train_val_test_split(prostate_data[:,:-1], prostate_data[:,-1], 0.1, 0.1, np.random.random_integers(31475))
#unpacking
x_train, x_val, x_test, y_train, y_val, y_test = split_data

print('Input length:', len(prostate_data))
print('Number of training samples:', len(x_train))
print('Number of validation samples:', len(x_val))
print('Number of testing samples:', len(x_test))

Input length: 97
Number of training samples: 77
Number of validation samples: 10
Number of testing samples: 10


/tmp/ipykernel_1269/3310943873.py:1: DeprecationWarning: This function is deprecated. Please call randint(1, 31475 + 1) instead
  split_data = train_val_test_split(prostate_data[:,:-1], prostate_data[:,-1], 0.1, 0.1, np.random.random_integers(31475))


In [1840]:
x_train = (x_train - x_train.mean(axis=0)) / x_train.std(axis=0)
x_val = (x_val - x_val.mean(axis=0)) / x_val.std(axis=0)
x_test = (x_test - x_test.mean(axis=0)) / x_test.std(axis=0)

In [1841]:
class LinearModel:
    def __init__(self):
        self.beta = None

    def fit(self, x,y):
        X = np.concatenate((np.ones((x.shape[0], 1)), x), axis = 1)
        self.beta = np.linalg.solve((X.T @ X),X.T @ y)
        return self.beta

    def predict(self,x, with_intercept = True):
        if with_intercept:
            X = np.concatenate((np.ones((x.shape[0], 1)),x), axis = 1)
            y_hat = X @ self.beta
        else:
            y_hat = x @ self.beta[1:]
        return y_hat

def mean_square_error(y, y_hat):
    return np.mean(np.square(np.array(y-y_hat)))

In [1842]:
linearModel = LinearModel()
beta = linearModel.fit(x_train, y_train)

#MSEs
test_error = mean_square_error(y_test, linearModel.predict(x_test))
train_error = mean_square_error(y_train, linearModel.predict(x_train))
print("Test MSE:", test_error)
beta

Test MSE: 0.32421657723522296


array([ 2.3487587 ,  0.69667922,  0.33136687, -0.16748868,  0.13416529,
        0.23096276, -0.13569651, -0.00646733,  0.14117769])

In [1843]:
def format_corr_table(corr_mat, coeff_names):
  '''Function to format the data into a correlation table'''

  corr_table = []
  for row_num in range(corr_mat.shape[0]):
    vals = list(corr_mat[row_num, :])
    vals = [str(round(val,3)) for val in vals]
    vals.insert(0,coeff_names[row_num])
    vals = vals [:row_num+1]
    corr_table.append(vals)

  return corr_table

In [1844]:
#Calculate the correlation coefficients
corr_mat = np.corrcoef(x_train.T)

#formatting the correlation table 
coeff_names = list(df_prostate.columns.values[:-1])
corr_table = format_corr_table(corr_mat, coeff_names)

#Fixing title
coeff_names.insert(0,'Coefficients')

#generate and print correlation table
corr = tabulate(corr_table[1:], headers=coeff_names, tablefmt='pretty')
print('     Table 1: Correlations of predictors in the prostate cancer data.')
print(corr)

     Table 1: Correlations of predictors in the prostate cancer data.
+--------------+--------+---------+-------+-------+-------+-------+---------+
| Coefficients | lcavol | lweight |  age  | lbph  |  svi  |  lcp  | gleason |
+--------------+--------+---------+-------+-------+-------+-------+---------+
|   lweight    | 0.353  |         |       |       |       |       |         |
|     age      | 0.274  |  0.347  |       |       |       |       |         |
|     lbph     | 0.138  |  0.377  | 0.35  |       |       |       |         |
|     svi      | 0.489  |  0.259  | 0.128 | -0.01 |       |       |         |
|     lcp      | 0.689  |  0.226  |  0.1  | 0.083 | 0.677 |       |         |
|   gleason    | 0.514  |  0.103  | 0.299 | 0.132 | 0.395 | 0.622 |         |
|    pgg45     | 0.507  |  0.173  | 0.28  | 0.094 | 0.508 | 0.699 |  0.81   |
+--------------+--------+---------+-------+-------+-------+-------+---------+


In [1845]:
def z_scorer(model, X, y):
    '''Returns standard error and z-score'''
    #extract metrics from x
    N = len(x_train)
    p = X.shape[1]

    #Using 3.8b to get the stdv
    sigma =  np.sqrt(np.sum((y - model.predict(X))**2)/(N-p-1))

    #since predict automatically adds the one column, we add it after here
    X = np.concatenate((np.ones((N, 1)),X), axis = 1)
    
    #implement equation 3.12 to get the z score
    sqrt_v = np.sqrt(np.diagonal(np.linalg.inv(X.T @ X)))
    standard_error = sigma*sqrt_v
    print(model.beta)
    z_score = model.beta/standard_error

    return standard_error, z_score

In [1846]:
def format_lin_summary_table(term, beta, s_e, z_scr, round_decimal = 2):
  '''Function to format the data into a summary table'''
  summary_mat = []
  #generate a list of lists, where the inner lists are rows
  for coeff_num in range(len(term)):
    row = list([term[coeff_num],
              round(beta[coeff_num], round_decimal),
              round(s_e[coeff_num], round_decimal),
              round(z_scr[coeff_num], round_decimal)])
    summary_mat.append(row)

  return summary_mat

In [1847]:
#coefficient analysis table
standard_error, Z_score = z_scorer(linearModel, x_train, y_train)

term = list(df_prostate.columns.values[:-1])
term.insert(0, 'Intercept')
summary_lst = format_lin_summary_table(term, beta, standard_error, Z_score)

summary_header = ['Term', 'Coefficient', 'Std. Error', 'Z Score']
summary = tabulate(summary_lst, headers = summary_header, tablefmt='pretty')
print('Table 2: Linear model fit to the prostate cancer data.')
print(summary)

[ 2.3487587   0.69667922  0.33136687 -0.16748868  0.13416529  0.23096276
 -0.13569651 -0.00646733  0.14117769]
Table 2: Linear model fit to the prostate cancer data.
+-----------+-------------+------------+---------+
|   Term    | Coefficient | Std. Error | Z Score |
+-----------+-------------+------------+---------+
| Intercept |    2.35     |    0.08    |  28.14  |
|  lcavol   |     0.7     |    0.12    |  5.59   |
|  lweight  |    0.33     |    0.1     |   3.3   |
|    age    |    -0.17    |    0.1     |  -1.67  |
|   lbph    |    0.13     |    0.1     |  1.41   |
|    svi    |    0.23     |    0.12    |  1.97   |
|    lcp    |    -0.14    |    0.16    |  -0.83  |
|  gleason  |    -0.01    |    0.15    |  -0.04  |
|   pgg45   |    0.14     |    0.16    |  0.87   |
+-----------+-------------+------------+---------+
